# Dictionary comprehension

Based on discussion from: https://stackoverflow.com/questions/52542742/why-is-this-loop-faster-than-a-dictionary-comprehension-for-creating-a-dictionar

## Is dict comprehension faster than creating dictionary with a loop?

In [1]:
a = {'a':'hi','b':'hey','c':'yo'}

In [2]:
%%timeit
b = {}
for k, v in a.items():
    b[v] = k

160 ns ± 0.601 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [3]:
%%timeit
b = {v: k for k, v in a.items()}

218 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


With a dummy exmaple it looks like dict comprehension is slower than looped version, 
which is a bit counterintuitive. However, the issue here is that this example is way too small to judge the performance. 
When you check the performance, make sure to run the test on a **good test case with enough data to show the differences.**


In [4]:
from random import choice, randrange
from string import ascii_lowercase as letters

In [5]:
def random_string():
    return ''.join([choice(letters) for _ in range(randrange(3, 15))])

In [6]:
test_dict = {random_string(): random_string() for _ in range(1000)}

In [7]:
%%timeit
b = {}
for k, v in test_dict.items():
    b[v] = k

39.4 µs ± 434 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
%%timeit
b = {v: k for k, v in test_dict.items()}

36.9 µs ± 402 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


With 1000 keys in test dictionary, dict comprehension is in fact slightly faster.

In [9]:
test_dict = {random_string(): random_string() for _ in range(100000)}

In [10]:
%%timeit
b = {}
for k, v in test_dict.items():
    b[v] = k

5.72 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%%timeit
b = {v: k for k, v in test_dict.items()}

5.66 ms ± 83.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


With 100 000 keys the differnece gets even bigger. So in practice, it is better to use dict comprehension, because:  
 - it is slightly faster than looped version for > 1000 keys
 - for smaller dictionaries perfomance is neglibable anyway

## Why is dict comprehension slower for small dictionaries?

What is different in execution of dict comprehension and looped version?
Can we find this differnece by looking at the disassembled Python bytecodes?

In [12]:
import dis

In [13]:
looped = '''\
... b = {}
... for k,v in test_dict.items():
...     b[v]=k
... '''

In [14]:
dis.dis(looped)

  1           0 BUILD_MAP                0
              2 STORE_NAME               0 (b)

  2           4 LOAD_NAME                1 (test_dict)
              6 LOAD_METHOD              2 (items)
              8 CALL_METHOD              0
             10 GET_ITER
        >>   12 FOR_ITER                16 (to 30)
             14 UNPACK_SEQUENCE          2
             16 STORE_NAME               3 (k)
             18 STORE_NAME               4 (v)

  3          20 LOAD_NAME                3 (k)
             22 LOAD_NAME                0 (b)
             24 LOAD_NAME                4 (v)
             26 STORE_SUBSCR
             28 JUMP_ABSOLUTE           12
        >>   30 LOAD_CONST               0 (None)
             32 RETURN_VALUE


In [15]:
dictcomp = '''b = {v: k for k, v in test_dict.items()}'''

In [16]:
dis.dis(dictcomp)

  1           0 LOAD_CONST               0 (<code object <dictcomp> at 0x106993f50, file "<dis>", line 1>)
              2 LOAD_CONST               1 ('<dictcomp>')
              4 MAKE_FUNCTION            0
              6 LOAD_NAME                0 (test_dict)
              8 LOAD_METHOD              1 (items)
             10 CALL_METHOD              0
             12 GET_ITER
             14 CALL_FUNCTION            1
             16 STORE_NAME               2 (b)
             18 LOAD_CONST               2 (None)
             20 RETURN_VALUE

Disassembly of <code object <dictcomp> at 0x106993f50, file "<dis>", line 1>:
  1           0 BUILD_MAP                0
              2 LOAD_FAST                0 (.0)
        >>    4 FOR_ITER                14 (to 20)
              6 UNPACK_SEQUENCE          2
              8 STORE_FAST               1 (k)
             10 STORE_FAST               2 (v)
             12 LOAD_FAST                2 (v)
             14 LOAD_FAST                1 (

The main difference comes from `MAKE_FUNCTION` and `CALL_FUNCTION` opcodes in the top-level bytecode for the dict comprehension

The `MAKE_FUNCTION` and `CALL_FUNCTION` opcodes the dict comphrension has to execute is dominates execution time for smaller dictionaries.

In [17]:
make_and_call = '(lambda i: None)(None)'

In [18]:
dis.dis(make_and_call)

  1           0 LOAD_CONST               0 (<code object <lambda> at 0x1068ffbe0, file "<dis>", line 1>)
              2 LOAD_CONST               1 ('<lambda>')
              4 MAKE_FUNCTION            0
              6 LOAD_CONST               2 (None)
              8 CALL_FUNCTION            1
             10 RETURN_VALUE

Disassembly of <code object <lambda> at 0x1068ffbe0, file "<dis>", line 1>:
  1           0 LOAD_CONST               0 (None)
              2 RETURN_VALUE


In [19]:
%%timeit
make_and_call

10.8 ns ± 0.0503 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


# List comprehension

What about the performance of list comprehensions?

In [20]:
def squares(values):
    lst = []
    for x in range(values):
        lst.append(x*x)
    return lst

In [21]:
%%timeit
squares(1000)

47.9 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
def squares_comp(values):
    return [x*x for x in range(values)]

In [23]:
%%timeit
squares_comp(1000)

30.4 µs ± 89.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


List comprehension is clearly fast. Can we understand it by comparing the bytecode?

In [24]:
dis.dis(squares)

  2           0 BUILD_LIST               0
              2 STORE_FAST               1 (lst)

  3           4 LOAD_GLOBAL              0 (range)
              6 LOAD_FAST                0 (values)
              8 CALL_FUNCTION            1
             10 GET_ITER
        >>   12 FOR_ITER                18 (to 32)
             14 STORE_FAST               2 (x)

  4          16 LOAD_FAST                1 (lst)
             18 LOAD_METHOD              1 (append)
             20 LOAD_FAST                2 (x)
             22 LOAD_FAST                2 (x)
             24 BINARY_MULTIPLY
             26 CALL_METHOD              1
             28 POP_TOP
             30 JUMP_ABSOLUTE           12

  5     >>   32 LOAD_FAST                1 (lst)
             34 RETURN_VALUE


In [25]:
dis.dis(squares_comp)

  2           0 LOAD_CONST               1 (<code object <listcomp> at 0x1069b8030, file "/var/folders/0g/n35wm48d37bcjss5tyrt9l040000gn/T/ipykernel_50231/2847316437.py", line 2>)
              2 LOAD_CONST               2 ('squares_comp.<locals>.<listcomp>')
              4 MAKE_FUNCTION            0
              6 LOAD_GLOBAL              0 (range)
              8 LOAD_FAST                0 (values)
             10 CALL_FUNCTION            1
             12 GET_ITER
             14 CALL_FUNCTION            1
             16 RETURN_VALUE

Disassembly of <code object <listcomp> at 0x1069b8030, file "/var/folders/0g/n35wm48d37bcjss5tyrt9l040000gn/T/ipykernel_50231/2847316437.py", line 2>:
  2           0 BUILD_LIST               0
              2 LOAD_FAST                0 (.0)
        >>    4 FOR_ITER                12 (to 18)
              6 STORE_FAST               1 (x)
              8 LOAD_FAST                1 (x)
             10 LOAD_FAST                1 (x)
             12 BIN

The `squares` function looks up the `append` method of the list in each iteration, and calls it.
The `append` function has to grow the list by one element each time itis called.

The list comprehension on the other hand doesn't have to do that work. Instead, python uses `LIST_APPEND` bytecode, which uses the C API to append a new element to the list, without having to do the lookup and a python call to the function.